In [ ]:
#安裝module到kaggle環境
!cp -r /kaggle/input/d/evanhsiao/indoor-location-navigation/* ./
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json
from io_f import read_data_file
from main import calibrate_magnetic_wifi_ibeacon_to_position, extract_magnetic_strength, extract_wifi_rssi, extract_ibeacon_rssi, extract_wifi_count
from compute_f import compute_step_positions
from visualize_f import visualize_trajectory, visualize_heatmap
import compute_f
import visualize_f
import keras as k
from keras import layers
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
np.random.seed(1)
from dataclasses import dataclass
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Functions
def test_data_to_test_x(ReadData):
    '''
    Retrive features from test data file to model
    
    Argument
    ReadData: Object from read_data_file function; dtype ReadData object
    
    Return
    test_x: A [features, sample#] matrix for model; dtype np.array
    '''
    acce_data = ReadData.acce
    acce_uncali_data = ReadData.acce_uncali
    gyro_data = ReadData.gyro
    gyro_uncali_data = ReadData.gyro_uncali
    magn_data = ReadData.magn
    magn_uncali_data = ReadData.magn_uncali
    ahrs_data = ReadData.ahrs
    wifi_data = ReadData.wifi
    ibeacon_data = ReadData.ibeacon
    waypoint_data = ReadData.waypoint
    
    test_x = np.zeros(((acce_data.shape[1]-1)*3 + 1, acce_data.shape[0]))
    for i in range(acce_data.shape[0]):
        timestep = acce_data[i, 0]
        acce = acce_data[i:i+1, 0:]
        magn = magn_data[i:i+1, 1:]
        ahrs = ahrs_data[i:i+1, 1:]
        test_x[:, i:i+1] = np.concatenate((acce, magn, ahrs), axis = 1).T
    return test_x
    
    
def location_sensor_filter(ReadData):
    '''
    Filter acce and magn data and reshape training data into shape:(features, samples) array; training label into shape:(locations, samples) array
    
    Arguments
    ReadData: A object from txt file; dtype ReadData object
    
    Returns
    data: A [features, sample#] matrix; dtype np.array
    label: A [waypoints, samples#] matrix; dtype np.array
    '''
    acce_data = ReadData.acce
    acce_uncali_data = ReadData.acce_uncali
    gyro_data = ReadData.gyro
    gyro_uncali_data = ReadData.gyro_uncali
    magn_data = ReadData.magn
    magn_uncali_data = ReadData.magn_uncali
    ahrs_data = ReadData.ahrs
    wifi_data = ReadData.wifi
    ibeacon_data = ReadData.ibeacon
    waypoint_data = ReadData.waypoint
    
    data = np.zeros(((acce_data.shape[1]-1)*3, acce_data.shape[0]))
    label = np.zeros((2, acce_data.shape[0])) 
    locations = compute_step_positions(acce_data, ahrs_data, waypoint_data)
    
    for i in range(acce_data.shape[0]):
        timestep = acce_data[i, 0]
        acce = acce_data[i:i+1, 1:]
        magn = magn_data[i:i+1, 1:]
        ahrs = ahrs_data[i:i+1, 1:]
        data[:, i:i+1] = np.concatenate((acce, magn, ahrs), axis = 1).T
        if timestep < locations[-1, 0]:
            index = np.where(locations[:, 0] > timestep)[0][0]
            if index == 0:
                upper_timestep = locations[1, 0]  
                lower_timestep = locations[0, 0]
                vector = locations[1, 1:] - locations[0, 1:]
                adjust = (lower_timestep - timestep) / (upper_timestep - lower_timestep)
                location = locations[0, 1:] - vector * adjust
                label[:, i] = location.T                
            else:
                upper_timestep = locations[index, 0]  
                lower_timestep = locations[index-1, 0]
                vector = locations[index, 1:] - locations[index-1, 1:]
                adjust = (timestep - lower_timestep) / (upper_timestep - lower_timestep)
                location = locations[index-1, 1:] + vector * adjust
                label[:, i] = location.T
        else:
            upper_timestep = locations[-1, 0]  
            lower_timestep = locations[-2, 0]
            vector = locations[-1, 1:] - locations[-2, 1:]
            adjust = (timestep - upper_timestep) / (upper_timestep - lower_timestep)
            location = locations[-1, 1:] + vector * adjust
            label[:, i] = location.T            
                
    return data, label

def location_training(model, train_paths, epo=1):
    '''
    Fit model over all train set.
    
    Arguments
    model: Customized Neural Network model; dtype k.model
    train_paths: All training data file paths; dtype list
    epo = Epoch# over all training set; dtype int
    
    Returns
    model: A trained model to predict waypoint; dtype k.model
    '''
    for i in range(epo):
        for step, path in enumerate(train_paths):
            try:
                path_data = read_data_file(path)
                data, label = location_sensor_filter(path_data)
                indices = np.random.permutation(data.shape[1])
                data = data[:, indices]
                label = label[:, indices]
                dev_x = data[:, -100:]
                dev_y = label[:, -100:]
                train_x = data[:, :-100]
                train_y = label[:, :-100]
                history = model.fit(train_x.T, train_y.T, batch_size=64, epochs = epo+1, validation_data = (dev_x.T, dev_y.T), verbose=0, shuffle=True)
                if step % 100 == 0:
                    print(f'Training loss: {history.history["loss"][0]}; Dev loss: {history.history["val_loss"][0]} at step {step+1}.')
            except:
                print(f'Traing faile at {path}.')
                continue
    return model

def site_floor_bssid_rssi_dic(submission_site_paths, rssi_threshold = -75):
    '''
    Record max rssi of bssid of each site-floor into a {site: {floor: {bssid: rssi}}} dictionary from training set.
    
    Arguments
    submission_site_paths: All paths in submission.csv file; dtype list
    rssi_threshold: filter weak signal; dtype int
    
    Returns
    max_rssi_dic: A {site: {floor: {bssid: rssi}}} dictionary which sites are needed in submission file; dtype dic
    '''
    max_rssi_dic = {}
    for path in submission_site_paths:
        path_list = path.split('/')
        site = path_list[-3]
        floor = path_list[-2]
        path_data = read_data_file(path)
        wifi_data = path_data.wifi
        try:
            for i in range(wifi_data.shape[0]):
                bssid = wifi_data[i, 2] 
                rssi = wifi_data[i, 3]
                if int(rssi) > rssi_threshold: #ignore weak signal
                    if site not in max_rssi_dic.keys():
                        max_rssi_dic[site] = {floor: {bssid: rssi}}
                    else:
                        if floor not in max_rssi_dic[site].keys():
                            max_rssi_dic[site][floor] = {bssid: rssi}
                        else:
                            if bssid not in max_rssi_dic[site][floor].keys():
                                max_rssi_dic[site][floor][bssid] = rssi
                            else:
                                if rssi > max_rssi_dic[site][floor][bssid]:
                                    max_rssi_dic[site][floor][bssid] = rssi
        except:
            print(f'There is an abnormal file: {path}')
    return max_rssi_dic

def floor_detect(ReadData, site, max_rssi_dic):
    '''
    Grade points for each floor to predict which floor on test path.
    
    Arguments
    Readdata: path data in test set; dtype: Readdata
    site: site of path data; dtype: str
    max_rssi_dic: A {site: {floor: {bssid: rssi}}} dictionary from training set; dtype dic
    
    Returns
    F: floor on test path; dtype int
    '''
    wifi_data = ReadData.wifi
    floor_score = {}
    F = 0
    for floor in max_rssi_dic[site].keys():
        floor_score[floor] = 0
    for i in range(wifi_data.shape[0]):
        test_bssid = wifi_data[i, 2]
        test_rssi = wifi_data[i, 3]
        for floor in max_rssi_dic[site].keys():
            for bssid in max_rssi_dic[site][floor].keys():
                if test_bssid == bssid and int(test_rssi) < int(max_rssi_dic[site][floor][bssid]):
                    floor_score[floor] += 1
                    break
    for floor, score in floor_score.items():
        if score == max(floor_score.values()):
            F = floor
            break   
    if F == '1F' or F == 'F1':
        F = 0
    elif F == '2F' or F == 'F2':
        F = 1
    elif F == '3F' or F == 'F3':
        F = 2
    elif F == '4F' or F == 'F4':
        F = 3
    elif F == '5F' or F == 'F5':
        F = 4
    elif F == '6F' or F == 'F6':
        F = 5
    elif F == '7F' or F == 'F7':
        F = 6
    elif F == '8F' or F == 'F8':
        F = 7
    elif F == 'B1' or F == '1B':
        F = -1
    elif F == 'B2' or F == '2B':
        F = -2
    elif F == 'B3' or F == '3B':
        F = -3
    elif F == 'B4' or F == '4B':
        F = -4
    return F
    
         

def MSE(x_pred, y_pred, f_pred, x_true, y_true, f_true, p=15):
    '''
    A customized loss function to evaluate model performance
    
    Arguments
    x_pred: x coordinate of waypoint; dtype np.array
    y_pred: y coordinate of waypoint; dtype np.array
    f_pred: floor of the site; dtype np.array
    x_true: x coordinate of waypoint; dtype np.array
    y_ture: y coordinate of waypoint; dtype np.array
    z_true: floor of the site; dtype list()
    p: a constant of floor penalty, set to 15 (always); dtype int

    Returns
    formula: value of loss function; dtype int
    '''
    
    N = len(x_true)
    formula = np.sqrt(np.power(x_pred - x_true, 2) + np.power(y_pred - y_true, 2))
    formula = formula + p * np.abs(f_pred - f_true)
    formula = formula.sum() / N
    
    return formula
def location_prediction_model(features):
    '''
    Simple Neural Network model.
    
    Arguments
    features: Input feature numbers; dtype int
    
    Returns
    model: A 3 hidden layers neural network model; dtype k.model
    '''
    model = k.Sequential(name='location_prediction')
    model.add(k.Input(shape=(features, )))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(2))
    model.summary()  
    return model

def submission_to_dic(csv):
    '''
    Return site dictionary {sites : {paths : [timesteps]}} for updating submission csv file.
    
    Arguments
    csv: Submission.csv file; dtype pd.DataFrame
    
    Returns
    site_dic: Dictiondary {sites : {paths : [timesteps]}}; dtype dic
    '''
    site_dic = {}
    for site_path in csv['site_path_timestamp']:
        site_path_list = site_path.split('_')
        site = site_path_list[0]
        path = site_path_list[1]
        timestep = site_path_list[2]
        
        if site not in site_dic.keys():
            site_dic[site] = {path: [timestep]}
        else:
            if path not in site_dic[site].keys():
                site_dic[site][path] = [timestep]
            else:
                site_dic[site][path].append(timestep)
    return site_dic
def submission_to_submission_paths(site_dic):
    '''
    Retrive paths where in submission.csv from train paths.
    
    Arguments
    site_dic: A {sites: {path: timestep}} dictionary from submission; dtype dic
    
    Returns
    submission_paths: paths where in submission.csv in training set; dtype list
    '''
    submission_paths = []
    for site in site_dic.keys():
        submission_paths += glob.glob(f'../input/indoor-location-navigation/train/{site}/*/*')
    return submission_paths

In [ ]:
#視覺化compute_step_positions function
path = f'../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt'
#read_data_file會將檔案路徑(path)回傳ReadData object，包含屬性
#TYPE_ACCELEROMETER: acce dtype np.array
#TYPE_ACCELEROMETER_UNCALIBRATED: acce_uncali dtype np.array
#TYPE_GYROSCOPE: gyro dtype np.array
#TYPE_GYROSCOPE_UNCALIBRATED: gyro_uncali dtype np.array
#TYPE_MAGNETIC_FIELD: magn 
#TYPE_MAGNETIC_FIELD_UNCALIBRATED: magn_uncali dtype np.array
#TYPE_ROTATION_VECTOR: ahrs dtype np.array
#TYPE_WIFI: wifi dtype np.array
#TYPE_BEACON: ibeacon dtype np.array
#TYPE_WAYPOINT: waypoint dtype np.array
#範例如下輸出前五筆TYPE_ACCELEROMETER [timestep, x, y, z]
example = read_data_file(path)
positions = compute_step_positions(example.acce, example.ahrs, example.waypoint)
test_waypoint = np.concatenate((example.waypoint[0:1], example.waypoint[-1:]), axis=0)
test_positions = compute_step_positions(example.acce, example.ahrs, test_waypoint)
print(f'Positions shape: {positions.shape}')
#紅點為example中貼上標籤的waypoint
#藍點為利用compute_step_positions function計算waypoint label之間移動的位置
#綠點為waypoint label為第一筆和最後一筆，可以發現誤差會愈來愈大，直到最後收斂到最後一筆waypoint
plt.figure(figsize=(16, 6))
plt.xlabel('X')
plt.ylabel('Y')
plt.scatter(positions[:, 1], positions[:, 2], color = 'blue')
plt.scatter(test_positions[:, 1], test_positions[:, 2], color = 'green', marker='*')
plt.scatter(example.waypoint[:, 1], example.waypoint[:, 2], color = 'red')



In [ ]:
test_waypoint.shape

In [ ]:
#視覺化site: 5a0546857ecc773753327266, floor:B1的圖面
file_path = f'../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt'
site = file_path.split('/')[-3]
floor = file_path.split('/')[-2]
path_id = file_path.split('/')[-1][:-4]
floor_info_filename = f'../input/indoor-location-navigation/metadata/{site}/{floor}/floor_info.json'
floor_plan_filename = f'../input/indoor-location-navigation/metadata/{site}/{floor}/floor_image.png'
with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]
path_data = read_data_file(file_path)
print('Visualizing ground truth positions...')
fig = visualize_trajectory(path_data.waypoint[:, 1:3], floor_plan_filename, width_meter, height_meter, title=path_id, show=True)



In [ ]:
#讀取資料以視覺化更多資訊
print('Visualizing more information...')
file_paths = glob.glob('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/*')
mwi_datas = calibrate_magnetic_wifi_ibeacon_to_position(file_paths)

In [ ]:
mwi_datas[(106.67530172033818, 162.3157427483535)]

In [ ]:
#視覺化site: 5a0546857ecc773753327266, floor:B1 所有training set內收集的路徑
step_positions = np.array(list(mwi_datas.keys()))
fig2 = visualize_trajectory(step_positions, floor_plan_filename, width_meter, height_meter, mode='markers', title='Step Position', show=True)

In [ ]:
#視覺化site: 5a0546857ecc773753327266, floor:B1 所有training set內收集的磁場強度
magnetic_strength = extract_magnetic_strength(mwi_datas)
heat_positions = np.array(list(magnetic_strength.keys()))
heat_values = np.array(list(magnetic_strength.values()))
fig3 = visualize_heatmap(heat_positions, heat_values, floor_plan_filename, width_meter, height_meter, colorbar_title='mu tesla', title='Magnetic Strength', show=True)

In [ ]:
#視覺化site: 5a0546857ecc773753327266, floor:B1 所有training set內收集的wifi_id: rssi強度
wifi_rssi = extract_wifi_rssi(mwi_datas)
print(f'This floor has {len(wifi_rssi.keys())} wifi aps')
ten_wifi_bssids = list(wifi_rssi.keys())[0:10]
print('Example 10 wifi ap bssids:\n')
for bssid in ten_wifi_bssids:
    print(bssid)
#target_wifi = input(f"Please input target wifi ap bssid:\n")
target_wifi = '0b64e537cc3d1818ec46f94f8dc14043a98d0089'
# target_wifi = '1e:74:9c:a7:b2:e4'
heat_positions = np.array(list(wifi_rssi[target_wifi].keys()))
heat_values = np.array(list(wifi_rssi[target_wifi].values()))[:, 0]
fig4 = visualize_heatmap(heat_positions, heat_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'Wifi: {target_wifi} RSSI', show=True)

In [ ]:
test_path = '../input/indoor-location-navigation/test/00ff0c9a71cc37a2ebdd0f05.txt'
test_data = read_data_file(test_path)
max_rssi = max([int(i) for i in test_data.wifi[:, 3]])
index = list(test_data.wifi[:, 3]).index(str(max_rssi))
bssid = test_data.wifi[index][2]
wifi_rssi_max_test = {}
for bssid, dic in wifi_rssi.items():
    max_rssi = max([array[0] for array in dic.values()])
    for waypoint, array in dic.items():
        if array[0] == max_rssi:
            wifi_rssi_max_test[bssid] = waypoint
            break
wifi_rssi_max_test.keys()
#找最近的

In [ ]:
#視覺化site: 5a0546857ecc773753327266, floor:B1 所有training set內收集的ibeacon_id: rssi強度
ibeacon_rssi = extract_ibeacon_rssi(mwi_datas)
print(f'This floor has {len(ibeacon_rssi.keys())} ibeacons')
ten_ibeacon_ummids = list(ibeacon_rssi.keys())[0:10]
print('Example 10 ibeacon UUID_MajorID_MinorIDs:\n')
for ummid in ten_ibeacon_ummids:
    print(ummid)
#target_ibeacon = input(f"Please input target ibeacon UUID_MajorID_MinorID:\n")
target_ibeacon = '89cb11b04122cef23388b0da06bd426c1f48a9b5_cfc84f0752adc96b489f71195d91a946c5f6d3e8_8159618423dfa22f1ca0b62543e2f18eef630ce8'
# target_ibeacon = 'FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10073_61418'
heat_positions = np.array(list(ibeacon_rssi[target_ibeacon].keys()))
heat_values = np.array(list(ibeacon_rssi[target_ibeacon].values()))[:, 0]
fig5 = visualize_heatmap(heat_positions, heat_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'iBeacon: {target_ibeacon} RSSI', show=True)

In [ ]:
#視覺化site: 5a0546857ecc773753327266, floor:B1 所有training set內收集的wifi數量
wifi_counts = extract_wifi_count(mwi_datas)
heat_positions = np.array(list(wifi_counts.keys()))
heat_values = np.array(list(wifi_counts.values()))
# filter out positions that no wifi detected
mask = heat_values != 0
heat_positions = heat_positions[mask]
heat_values = heat_values[mask]
fig6 = visualize_heatmap(heat_positions, heat_values, floor_plan_filename, width_meter, height_meter, colorbar_title='number', title=f'Wifi Count', show=True)

In [ ]:
'''
#Model example
data, label = location_sensor_filter(example)
indices = np.random.permutation(data.shape[1])
data = data[:, indices]
label = label[:, indices]
dev_x = data[:, -200:]
dev_y = label[:, -200:]
train_x = data[:, :-200]
train_y = label[:, :-200]
print(f'Train_x: {train_x.shape}')
print(f'Train_y: {train_y.shape}')
print(f'Dev_x: {dev_x.shape}')
print(f'Dev_y: {dev_y.shape}')
model = location_prediction_model(train_x.shape[0])
model.compile(optimizer='adam', loss='mae')
history = model.fit(train_x.T, train_y.T, batch_size=64, epochs = 10, validation_data = (dev_x.T, dev_y.T), verbose=0, shuffle=True)

plt.figure(figsize=(20, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Dev'], loc='upper left')
plt.show()
'''

In [ ]:
#訓練模型
#用Glob module回傳所有train/test檔案路徑到串列變數中
train_paths = glob.glob('../input/indoor-location-navigation/train/*/*/*')
test_paths = glob.glob('../input/indoor-location-navigation/test/*')
site_paths = glob.glob('../input/indoor-location-navigation/metadata/*')
submission = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
submission_site_dic = submission_to_dic(submission)
print("No. Files in Train: {:,}".format(len(train_paths)), "\n" +
      "No. Files in Test: {:,}".format(len(test_paths)), "\n" +
      "Total Sites (metadata): {:,}".format(len(site_paths)))

#預測floor使用的函數執行時間過長，將運算結果存到.json檔供下次讀取
#submission_site_paths = submission_to_submission_paths(submission_site_dic)
#max_rssi_dic = site_floor_bssid_rssi_dic(submission_site_paths)
#with open('max_rssi_dic.json', 'w', encoding='utf-8') as f:
#    json.dump(max_rssi_dic, f)

with open('../input/d/evanhsiao/indoor-location-navigation/max_rssi_dic.json', 'r', encoding='utf-8') as f:
    max_rssi_dic = json.load(f)
'''
#訓練模型
model = location_prediction_model(train_x.shape[0])
model.compile(optimizer='adam', loss='mae')

trained_model = location_training(model, np.random.choice(train_paths, 1000))
'''
#測試floor detect在training data的表現
acc = {}
for site in submission_site_dic:
    score = 0
    path_datas = glob.glob(f'../input/indoor-location-navigation/train/{site}/*/*')
    n = len(path_datas)
    for path in path_datas:
        ans = path.split('/')[-2]
        if ans == '1F' or ans == 'F1':
            ans = 0
        elif ans == '2F' or ans == 'F2':
            ans = 1
        elif ans == '3F' or ans == 'F3':
            ans = 2
        elif ans == '4F' or ans == 'F4':
            ans = 3
        elif ans == '5F' or ans == 'F5':
            ans = 4
        elif ans == '6F' or ans == 'F6':
            ans = 5
        elif ans == '7F' or ans == 'F7':
            ans = 6
        elif ans == '8F' or ans == 'F8':
            ans = 7
        elif ans == 'B1' or ans == '1B':
            ans = -1
        elif ans == 'B2' or ans == '2B':
            ans = -2
        elif ans == 'B3' or ans == '3B':
            ans = -3
        elif ans == 'B4' or ans == '4B':
            ans = -4                 
        test_data = read_data_file(path)
        F = floor_detect(test_data, site, max_rssi_dic)
        if F == ans:
            score += 1
    acc[site] = score/n
acc

In [ ]:
'''
#預測結果並將結果更新到submission.csv檔
for site, path_dic in submission_site_dic.items():
    for path, timesteps in path_dic.items():
        path_data = f'../input/indoor-location-navigation/test/{path}.txt'
        test_data = read_data_file(path_data)
        F = floor_detect(test_data, site, max_rssi_dic)
        test_x = test_data_to_test_x(test_data)
        predictions = trained_model.predict(test_x.T[:, 1:])
        predictions_with_timestep = np.concatenate((test_x.T[:,0:1], predictions), axis=1)
        for timestep in timesteps:
            target = site + '_' + path + '_' + timestep
            int_timestep = int(timestep)
            if int_timestep < predictions_with_timestep[-1, 0]:
                index = np.where(predictions_with_timestep[:, 0] > int_timestep)[0][0]
                if index == 0:
                    location = predictions_with_timestep[0, 1:]
                    vector = predictions_with_timestep[1, 1:] - predictions_with_timestep[0, 1:]
                    location -= vector * (predictions_with_timestep[0,0] - int_timestep)/(predictions_with_timestep[1,0] - predictions_with_timestep[0,0])
                    submission.loc[submission['site_path_timestamp'] == target, 'x'] = location[0]
                    submission.loc[submission['site_path_timestamp'] == target, 'y'] = location[1]
                    submission.loc[submission['site_path_timestamp'] == target, 'floor'] = F
                else:
                    location = predictions_with_timestep[index-1, 1:]
                    vector = predictions_with_timestep[index, 1:] - predictions_with_timestep[index-1, 1:]
                    location += vector * (int_timestep - predictions_with_timestep[index-1,0])/(predictions_with_timestep[index,0] - predictions_with_timestep[index-1,0])
                    submission.loc[submission['site_path_timestamp'] == target, 'x'] = location[0]
                    submission.loc[submission['site_path_timestamp'] == target, 'y'] = location[1]
                    submission.loc[submission['site_path_timestamp'] == target, 'floor'] = F
            else:
                location = predictions_with_timestep[-1, 1:]
                vector = predictions_with_timestep[-1, 1:] - predictions_with_timestep[-2, 1:]
                location += vector * (int_timestep - predictions_with_timestep[index-1,0])/(predictions_with_timestep[-1,0] - predictions_with_timestep[index-2,0])
                submission.loc[submission['site_path_timestamp'] == target, 'x'] = location[0]
                submission.loc[submission['site_path_timestamp'] == target, 'y'] = location[1]
                submission.loc[submission['site_path_timestamp'] == target, 'floor'] = F
submission
'''

In [ ]:
'''
submission.to_csv('20210309_submission.csv', index=False)
print("Your submission was successfully saved!")
'''